# Extract Precompute Weights

In [1]:
# Load fine-tuned model
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("gokuls/BERT-tiny-emotion-intent")
model = AutoModelForSequenceClassification.from_pretrained("gokuls/BERT-tiny-emotion-intent")

model.eval()
model.double()

/Users/tonyma/code/FHE-BERT-Tiny-Emotion/env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-1

## Load Dataset

In [2]:
# Load train dataset
from datasets import load_dataset

train_dataset = load_dataset("dair-ai/emotion", split="train")

In [3]:
# Check the max length of the text
# Tokenize the dataset
def tokenize_function(example):
    return tokenizer(example['text'], truncation=False, padding=False)

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
max_length = max(len(sample) for sample in tokenized_train_dataset['input_ids'])
print(f"Max length: {max_length}")

Max length: 87


## Process

In [ ]:
import math
import torch
import numpy as np
from tqdm import tqdm

mean_distribution_0_0 = []
var_distribution_0_0 = []
vy_distribution_0_0 = [[] for _ in range(128)]
normbias_distribution_0_0 = []

mean_distribution_0_1 = []
var_distribution_0_1 = []
vy_distribution_0_1 = [[] for _ in range(128)]
normbias_distribution_0_1 = []

mean_distribution_1_0 = []
var_distribution_1_0 = []
vy_distribution_1_0 = [[] for _ in range(128)]
normbias_distribution_1_0 = []

mean_distribution_1_1 = []
var_distribution_1_1 = []
vy_distribution_1_1 = [[] for _ in range(128)]
normbias_distribution_1_1 = []

for sentence in tqdm(train_dataset['text']):
    text = "[CLS] " + sentence + " [SEP]"

    tokenized = tokenizer(text)
    tokenized_text = tokenizer.tokenize(text)
    segments_ids = [1] * len(tokenized_text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    
    # Embeddings
    x = model.bert.embeddings(tokens_tensor, segments_tensors)
    original_input_tensor = x.double()
    input_tensor = x.double()

    # Self-Attention
    fin = model.bert.encoder.layer[0].attention.self(x)[0].double()

    w_output_dense = model.bert.encoder.layer[0].attention.output.dense.weight.clone().detach().double().transpose(0, 1)
    b_output_dense = model.bert.encoder.layer[0].attention.output.dense.bias.clone().detach().double()

    fin2 = torch.matmul(fin, w_output_dense) + b_output_dense
    fin2_backup = fin2.clone()
    fin2_backup = fin2_backup + original_input_tensor

    mean_0_0 = []
    var_0_0 = []

    fin3_whole = []
    for i in range(len(original_input_tensor.squeeze())):
        fin2 = fin2_backup.squeeze()[i]

        current_mean = torch.mean(fin2.squeeze()).item()
        epsilon = model.bert.encoder.layer[0].attention.output.LayerNorm.eps
        current_var = torch.var(fin2.squeeze(), unbiased=False).item()
        current_inv_sqrt_var = 1 / math.sqrt(current_var + epsilon)

        # Save mean and variance
        mean_0_0.append(current_mean)
        var_0_0.append(current_inv_sqrt_var)

        fin3_corr = (fin2.squeeze().detach() - current_mean) * current_inv_sqrt_var

        w_output_layernorm = model.bert.encoder.layer[0].attention.output.LayerNorm.weight.clone().detach().double().unsqueeze(0)
        b_output_layernorm = model.bert.encoder.layer[0].attention.output.LayerNorm.bias.clone().detach().double()

        # Calculate vy
        vy = w_output_layernorm.squeeze(0) * current_inv_sqrt_var
        
        # Save vy & normbias
        normbias = b_output_layernorm
        vy_distribution_0_0[i].append(np.array(vy))
        normbias_distribution_0_0.append(np.array(normbias))

        fin3_corr = fin3_corr * w_output_layernorm + b_output_layernorm
        fin3_whole.append(fin3_corr.detach())

    mean_distribution_0_0.append(np.array(mean_0_0))
    var_distribution_0_0.append(np.array(var_0_0))

    fin3_whole = torch.cat(tuple(fin3_whole), 0).unsqueeze(0)
    input_tensor = fin3_whole
    fin_4 = torch.matmul(fin3_whole, model.bert.encoder.layer[0].intermediate.dense.weight.transpose(0, 1).double()) + model.bert.encoder.layer[0].intermediate.dense.bias

    fin_5 = torch.nn.functional.gelu(fin_4)
    fin_6 = torch.matmul(fin_5, model.bert.encoder.layer[0].output.dense.weight.transpose(0, 1).double()) + model.bert.encoder.layer[0].output.dense.bias
    fin_6 = fin_6 + fin3_whole

    mean_0_1 = []
    var_0_1 = []

    fin7_whole = []
    for i in range(len(input_tensor.squeeze())):
        fin7 = fin_6.squeeze()[i]

        current_mean = torch.mean(fin7.squeeze()).item()
        epsilon = model.bert.encoder.layer[0].output.LayerNorm.eps
        current_var = torch.var(fin7.squeeze(), unbiased=False).item()
        current_inv_sqrt_var = 1 / math.sqrt(current_var + epsilon)

        # Save mean and variance
        mean_0_1.append(np.array(current_mean))
        var_0_1.append(np.array(current_inv_sqrt_var))

        fin7_corr = (fin7.squeeze().detach() - current_mean) * current_inv_sqrt_var

        w_output_layernorm = model.bert.encoder.layer[0].output.LayerNorm.weight.clone().detach().double().unsqueeze(0)
        b_output_layernorm = model.bert.encoder.layer[0].output.LayerNorm.bias.clone().detach().double()

        # Calculate vy
        vy = w_output_layernorm.squeeze(0) * current_inv_sqrt_var

        # Save vy & normbias
        normbias = b_output_layernorm
        vy_distribution_0_1[i].append(np.array(vy))
        normbias_distribution_0_1.append(np.array(normbias))

        fin7_corr = fin7_corr * w_output_layernorm + b_output_layernorm
        fin7_whole.append(fin7_corr.detach())
        

    mean_distribution_0_1.append(np.array(mean_0_1))
    var_distribution_0_1.append(np.array(var_0_1))

    fin7_whole = torch.cat(tuple(fin7_whole), 0).unsqueeze(0)
    original_input_tensor = fin7_whole
    input_tensor = fin7_whole

    fin = model.bert.encoder.layer[1].attention.self(fin7_whole)[0].double()

    w_output_dense = model.bert.encoder.layer[1].attention.output.dense.weight.clone().detach().double().transpose(0, 1)
    b_output_dense = model.bert.encoder.layer[1].attention.output.dense.bias.clone().detach().double()

    fin2 = torch.matmul(fin, w_output_dense) + b_output_dense
    fin2_backup = fin2.clone()
    fin2_backup = fin2_backup + original_input_tensor

    mean_1_0 = []
    var_1_0 = []

    fin3_whole = []
    for i in range(len(original_input_tensor.squeeze())):
        fin2 = fin2_backup.squeeze()[i]

        current_mean = torch.mean(fin2.squeeze()).item()
        epsilon = model.bert.encoder.layer[1].attention.output.LayerNorm.eps
        current_var = torch.var(fin2.squeeze(), unbiased=False).item()
        current_inv_sqrt_var = 1 / math.sqrt(current_var + epsilon)

        mean_1_0.append(current_mean)
        var_1_0.append(current_inv_sqrt_var)

        fin3_corr = (fin2.squeeze() - current_mean) * current_inv_sqrt_var

        w_output_layernorm = model.bert.encoder.layer[1].attention.output.LayerNorm.weight.clone().detach().double().unsqueeze(0)
        b_output_layernorm = model.bert.encoder.layer[1].attention.output.LayerNorm.bias.clone().detach().double()

        # Calculate vy
        vy = w_output_layernorm.squeeze(0) * current_inv_sqrt_var

        # Save vy & normbias
        normbias = b_output_layernorm
        vy_distribution_1_0[i].append(np.array(vy))
        normbias_distribution_1_0.append(np.array(normbias))

        fin3_corr = fin3_corr * w_output_layernorm + b_output_layernorm
        fin3_whole.append(fin3_corr)

    mean_distribution_1_0.append(np.array(mean_1_0))
    var_distribution_1_0.append(np.array(var_1_0))

    fin3_whole = torch.cat(tuple(fin3_whole), 0).unsqueeze(0)
    input_tensor = fin3_whole
    fin_4 = torch.matmul(fin3_whole, model.bert.encoder.layer[1].intermediate.dense.weight.transpose(0, 1).double()) + model.bert.encoder.layer[1].intermediate.dense.bias

    fin_5 = torch.nn.functional.gelu(fin_4)
    fin_6 = torch.matmul(fin_5, model.bert.encoder.layer[1].output.dense.weight.transpose(0, 1).double()) + model.bert.encoder.layer[1].output.dense.bias
    fin_6 = fin_6 + fin3_whole
    
    mean_1_1 = []
    var_1_1 = []

    fin7_whole = []
    for i in range(len(input_tensor.squeeze())):
        fin7 = fin_6.squeeze()[i]

        current_mean = torch.mean(fin7.squeeze()).item()
        epsilon = model.bert.encoder.layer[1].output.LayerNorm.eps
        current_var = torch.var(fin7.squeeze(), unbiased=False).item()
        current_inv_sqrt_var = 1 / math.sqrt(current_var + epsilon)

        mean_1_1.append(current_mean)
        var_1_1.append(current_inv_sqrt_var)

        fin7_corr = (fin7.squeeze() - current_mean) * current_inv_sqrt_var

        w_output_layernorm = model.bert.encoder.layer[1].output.LayerNorm.weight.clone().detach().double().unsqueeze(0)
        b_output_layernorm = model.bert.encoder.layer[1].output.LayerNorm.bias.clone().detach().double()
        
        # Calculate vy
        vy = w_output_layernorm.squeeze(0) * current_inv_sqrt_var
        
        # Save vy & normbias
        normbias = b_output_layernorm
        vy_distribution_1_1[i].append(np.array(vy))
        normbias_distribution_1_1.append(np.array(normbias))
        
        fin7_corr = fin7_corr * w_output_layernorm + b_output_layernorm
        fin7_whole.append(fin7_corr.unsqueeze(0))

    mean_distribution_1_1.append(np.array(mean_1_1))
    var_distribution_1_1.append(np.array(var_1_1))

    fin7_whole = torch.cat(tuple(fin7_whole), 0).unsqueeze(0)

  0%|          | 0/16000 [00:00<?, ?it/s]/var/folders/3t/qv8zrd4d5t12dqxjq4rxf56r0000gn/T/ipykernel_44390/1459998300.py:77: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  vy_distribution_0_0[i].append(np.array(vy))
/var/folders/3t/qv8zrd4d5t12dqxjq4rxf56r0000gn/T/ipykernel_44390/1459998300.py:78: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  normbias_distribution_0_0.append(np.array(normbias))
/var/folders/3t/qv8zrd4d5t12dqxjq4rxf56r0000gn/T/ipykernel_44390/1459998300.py:120: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  vy_distribution_0_1[i].append(np.array(vy))
/var/folders/3t/qv8zrd4d5t12dqxjq4rxf56r0000gn/T/ipyker

In [5]:
# Compute average mean, var, normbias
precomputed_mean_0_0 = []
precomputed_var_0_0 = []
precomputed_mean_0_1 = []
precomputed_var_0_1 = []
precomputed_mean_1_0 = []
precomputed_var_1_0 = []
precomputed_mean_1_1 = []
precomputed_var_1_1 = []

precomputed_normbias_0_0 = []
precomputed_normbias_0_1 = []
precomputed_normbias_1_0 = []
precomputed_normbias_1_1 = []

current_mean_distribution = mean_distribution_0_0
current_var_distribution = var_distribution_0_0
current_normbias_distribution = normbias_distribution_0_0

for layer in range(4):
    print(f"Processing layer {layer}")
    # Calculate mean, var for each token position over the dataset
    #max_length = max(len(sample) for sample in current_mean_distribution)
    # Initilization
    total_means = np.zeros(128)
    total_vars = np.zeros(128)
    total_normbias = np.zeros(128)
    counts = np.zeros(128)

    for sample in current_mean_distribution:
        for i, value in enumerate(sample):
            if i < 128: # up to 128 positions
                total_means[i] += value
                
                counts[i] += 1
    
    for sample in current_var_distribution:
        for i, value in enumerate(sample):
            if i < 128:
                total_vars[i] += value

    # Calculate average mean/var for each position
    average_means = np.zeros(128)
    average_vars = np.zeros(128)
    for i in range(128):
        if counts[i] > 0:
            average_means[i] = total_means[i] / counts[i]
            average_vars[i] = total_vars[i] / counts[i]
    
    # Calculate average normbias
    average_normbias = np.mean(current_normbias_distribution, axis=0)

    if layer == 0:
        precomputed_mean_0_0.append(average_means)
        precomputed_var_0_0.append(average_vars)
        precomputed_normbias_0_0.append(average_normbias)
        current_mean_distribution = mean_distribution_0_1
        current_var_distribution = var_distribution_0_1
        current_normbias_distribution = normbias_distribution_0_1
    elif layer == 1:
        precomputed_mean_0_1.append(average_means)
        precomputed_var_0_1.append(average_vars)
        precomputed_normbias_0_1.append(average_normbias)
        current_mean_distribution = mean_distribution_1_0
        current_var_distribution = var_distribution_1_0
        current_normbias_distribution = normbias_distribution_1_0
    elif layer == 2:
        precomputed_mean_1_0.append(average_means)
        precomputed_var_1_0.append(average_vars)
        precomputed_normbias_1_0.append(average_normbias)
        current_mean_distribution = mean_distribution_1_1
        current_var_distribution = var_distribution_1_1
        current_normbias_distribution = normbias_distribution_1_1
    elif layer == 3:
        precomputed_mean_1_1.append(average_means)
        precomputed_var_1_1.append(average_vars)
        precomputed_normbias_1_1.append(average_normbias)

Processing layer 0
Processing layer 1
Processing layer 2
Processing layer 3


In [6]:
# Compute average vy
def compute_vy_mean(vy_distribution):
    hidden_size = model.config.hidden_size
    mean_vy = []

    for i in range(128):
        if len(vy_distribution[i]) > 0:
            stacked_vy = np.stack(vy_distribution[i], axis=0)
            mean_vy_i = np.mean(stacked_vy, axis=0)
        else:
            mean_vy_i = np.zeros(hidden_size)
        mean_vy.append(mean_vy_i)
    
    return np.array(mean_vy)

precomputed_vy_0_0 = compute_vy_mean(vy_distribution_0_0)
precomputed_vy_0_1 = compute_vy_mean(vy_distribution_0_1)
precomputed_vy_1_0 = compute_vy_mean(vy_distribution_1_0)
precomputed_vy_1_1 = compute_vy_mean(vy_distribution_1_1)

precomputed_vy_0_0 = precomputed_vy_0_0.T
precomputed_vy_0_1 = precomputed_vy_0_1.T
precomputed_vy_1_0 = precomputed_vy_1_0.T
precomputed_vy_1_1 = precomputed_vy_1_1.T

## Save

In [7]:
import os

os.makedirs('../emotion-precompute', exist_ok=True)

np.savetxt("../emotion-precompute/layer0_selfoutput_mean.txt", precomputed_mean_0_0, delimiter='\n')
np.savetxt("../emotion-precompute/layer0_output_mean.txt", precomputed_mean_0_1, delimiter='\n')
np.savetxt("../emotion-precompute/layer1_selfoutput_mean.txt", precomputed_mean_1_0, delimiter='\n')
np.savetxt("../emotion-precompute/layer1_output_mean.txt", precomputed_mean_1_1, delimiter='\n')

np.savetxt("../emotion-precompute/layer0_selfoutput_var.txt", precomputed_var_0_0, delimiter='\n')
np.savetxt("../emotion-precompute/layer0_output_var.txt", precomputed_var_0_1, delimiter='\n')
np.savetxt("../emotion-precompute/layer1_selfoutput_var.txt", precomputed_var_1_0, delimiter='\n')
np.savetxt("../emotion-precompute/layer1_output_var.txt", precomputed_var_1_1, delimiter='\n')

np.savetxt("../emotion-precompute/layer0_selfoutput_vy.txt", precomputed_vy_0_0, delimiter=',')
np.savetxt("../emotion-precompute/layer0_output_vy.txt", precomputed_vy_0_1, delimiter=',')
np.savetxt("../emotion-precompute/layer1_selfoutput_vy.txt", precomputed_vy_1_0, delimiter=',')
np.savetxt("../emotion-precompute/layer1_output_vy.txt", precomputed_vy_1_1, delimiter=',')

np.savetxt("../emotion-precompute/layer0_selfoutput_normbias.txt", precomputed_normbias_0_0, delimiter='\n')
np.savetxt("../emotion-precompute/layer0_output_normbias.txt", precomputed_normbias_0_1, delimiter='\n')
np.savetxt("../emotion-precompute/layer1_selfoutput_normbias.txt", precomputed_normbias_1_0, delimiter='\n')
np.savetxt("../emotion-precompute/layer1_output_normbias.txt", precomputed_normbias_1_1, delimiter='\n')